In [ ]:
import numpy as np
import matplotlib.pyplot as plt

## Defining Parameters:

The variant number 9 was chosen.

In [ ]:
#Defining Parameters:

youngs_modulus = 200e3#e9 #PA
poisson_ratio = 0.30
sigmaY = 400#e6 #Pa
r_in = 20#e-6 #m
r_out = 100#e-6 #m
epsilon_v = 0.001
final_time = 1 #seconds
delta_t = 0.1 #seconds
time_steps = 10
number_of_elements = 10
number_of_nodes = number_of_elements + 1


## Derived parameters:

In [ ]:
#Derived parameters:

lamda = (poisson_ratio * youngs_modulus)/((1 - 2*poisson_ratio)*(1+poisson_ratio))
mu  = youngs_modulus/ (2*(1+poisson_ratio))
epsilon_v_init = (1+poisson_ratio)*(sigmaY/youngs_modulus)

## Initializing all the measurable quantities:

In [ ]:
#time intervals
number_of_timesteps = 10

all_times = np.linspace(0 , 1 , number_of_timesteps)

#Initializing all the measurable quantities:

u = np.zeros((len(all_times),number_of_nodes,1)) #Displacements at every node and every time-step

epsilon_plastic = np.zeros((len(all_times),3,number_of_elements))  #Epsilon_Plastic for every element and in each timestep

epsilon_3D = np.zeros((len(all_times),3,number_of_elements)) # Strain in Voigt Notation for the given problem is 3x1 Matrix

stress_3D = np.zeros((len(all_times),3,number_of_elements)) # Stress in Voigt Noation for the given problem is 3x1 Matrix

Material_Stiffness = np.zeros((3,3)) #Stress = Stiffness x Strain --> C = 3x3 Matrix

Kt = np.zeros((number_of_nodes,number_of_nodes))

F_int = np.zeros((number_of_nodes,1))

F_ext = np.zeros((number_of_nodes,1))

delta_u = np.zeros((number_of_nodes,1))


## Define Material Routine :

In [ ]:
def material_routine(epsilon, epsilon_plastic_old, lamda, mu , sigmaY, youngs_modulus):

    C_ijkl = np.array([(2*mu + lamda), lamda, lamda,lamda,(2*mu + lamda), lamda, lamda,lamda,(2*mu + lamda)]).reshape(3,3)

    sigma_trial = np.dot(C_ijkl,(epsilon - epsilon_plastic_old))   #3x1 Matrix
    
    sigma_trial_dev = sigma_trial - np.sum(sigma_trial)/3          #3x1 Matrix
    
    sigma_trial_eq = np.asscalar(np.sqrt((1.5*np.dot((np.transpose(sigma_trial_dev)),sigma_trial_dev)))) #Scalar

    C_t = np.zeros((3,3))   # Algorithmically Consistent Tangent Stiffness Matrix
    
    if (np.sum(epsilon)<epsilon_v_init):
        print("******************ELASTIC REGION*********************")
    else:
        print("******************PLASTIC REGION*********************")

    #Yield Condition ------> Check for Elastic and Plastic Case 
    
    if (sigma_trial_eq - sigmaY) < 0:
        print("(sigma_trial_eq - sigmaY) < 0 -----------------------------------This is in Elastic Region")
        
        plastic_multiplier = 0
        
        C_t = np.array([(2*mu + lamda), lamda, lamda,lamda,(2*mu + lamda), lamda, lamda,lamda,(2*mu + lamda)]).reshape(3,3)
        
        sigma_new = 2*mu*(epsilon - epsilon_plastic_old) + lamda*np.sum(epsilon - epsilon_plastic_old)
        
        epsilon_plastic_new = epsilon_plastic_old
        
    else :
        print("(sigma_trial_eq - sigmaY) >0 --------------------------------------This is in  Plastic Region")
        
        plastic_multiplier = sigma_trial_eq /(3*mu + sigmaY)

        sigma_new = np.sum(sigma_trial)/3 + ((sigma_trial_eq - 3*mu*plastic_multiplier)/sigma_trial_eq)*sigma_trial_dev

    
        C_t[0][0] = ((3*lamda + 2*mu)/3 + (4*mu*(sigma_trial_eq - 3*mu*plastic_multiplier)/(3*sigma_trial_eq))) - (3*mu*sigma_trial_dev[0,0]*sigma_trial_dev[0,0])
        C_t[1][1] = ((3*lamda + 2*mu)/3 + (4*mu*(sigma_trial_eq - 3*mu*plastic_multiplier)/(3*sigma_trial_eq))) - (3*mu*sigma_trial_dev[1,0]*sigma_trial_dev[1,0])
        C_t[2][2] = ((3*lamda + 2*mu)/3 + (4*mu*(sigma_trial_eq - 3*mu*plastic_multiplier)/(3*sigma_trial_eq))) - (3*mu*sigma_trial_dev[2,0]*sigma_trial_dev[2,0])
        C_t[0][1] = ((3*lamda + 2*mu)/3 - (2*mu*(sigma_trial_eq - 3*mu*plastic_multiplier)/(3*sigma_trial_eq))) - (3*mu*sigma_trial_dev[0,0]*sigma_trial_dev[1,0])
        C_t[0][2] = ((3*lamda + 2*mu)/3 - (2*mu*(sigma_trial_eq - 3*mu*plastic_multiplier)/(3*sigma_trial_eq))) - (3*mu*sigma_trial_dev[0,0]*sigma_trial_dev[2,0])
        C_t[1][0] = ((3*lamda + 2*mu)/3 - (2*mu*(sigma_trial_eq - 3*mu*plastic_multiplier)/(3*sigma_trial_eq))) - (3*mu*sigma_trial_dev[1,0]*sigma_trial_dev[0,0])
        C_t[1][2] = ((3*lamda + 2*mu)/3 - (2*mu*(sigma_trial_eq - 3*mu*plastic_multiplier)/(3*sigma_trial_eq))) - (3*mu*sigma_trial_dev[1,0]*sigma_trial_dev[2,0])
        C_t[2][0] = ((3*lamda + 2*mu)/3 - (2*mu*(sigma_trial_eq - 3*mu*plastic_multiplier)/(3*sigma_trial_eq))) - (3*mu*sigma_trial_dev[2,0]*sigma_trial_dev[0,0])
        C_t[2][1] = ((3*lamda + 2*mu)/3 - (2*mu*(sigma_trial_eq - 3*mu*plastic_multiplier)/(3*sigma_trial_eq))) - (3*mu*sigma_trial_dev[2,0]*sigma_trial_dev[1,0])
        
        epsilon_plastic_new = epsilon_plastic_old + plastic_multiplier*np.sign(sigma_new)
    
    return sigma_new, C_t, epsilon_plastic_new


## Define Element Routine:

In [ ]:
#Defining my Element Routine:

def elementroutine(r,lamda, mu,u_element, epsilon_plastic,sigmaY,youngs_modulus):
    length_of_element = r[1]-r[0]
    jacobian = length_of_element/2
    weight = 2
    gauss_point = 0
    N1 = (1-gauss_point)/2
    N2 = (1+gauss_point)/2
    
    B = np.array([-1/(2*jacobian), 1/(2*jacobian), N1/(N1*r[0] + N2*r[1]), N2/(N1*r[0] + N2*r[1]), N1/(N1*r[0] + N2*r[1]), N2/(N1*r[0] + N2*r[1])]).reshape(3,2)
    
    epsilon = np.dot(B, u_element)

    stress_new , Material_Stiffness1, strain_plastic = material_routine(epsilon, epsilon_plastic, lamda, mu,sigmaY,youngs_modulus)
    
    Kt_element = weight * ((B.transpose()).dot(Material_Stiffness1).dot(B)) * (N1*r[0] + N2*r[1])**2 * jacobian
    
    F_int = Kt_element@u_element
    
    #F_ext = (N1*r[0] + N2*r[1])*stress_new[0]*(np.array([ N1,N2]).reshape(2,1))
    return Kt_element, F_int, stress_new, strain_plastic

## Discretizaton :

In [ ]:
#Defining the elements:
r = np.zeros((number_of_elements,2)) 

#ratio of element sizes at outer and inner radius
meshrefinementfactor = 5

#ratio between element sizes of subsequent elements for a geometric series
q = meshrefinementfactor**(1./(number_of_elements-1))

#size of first element
element_1 =(r_out-r_in)*(1-q)/(1-meshrefinementfactor*q)

temp_r = r_in

for i in range(number_of_elements):
    r[i,0] = temp_r
    r[i,1] = temp_r + element_1
    temp_r = r[i,1]
    element_1 = element_1*q

## Assignment/ Assembling Matrix:

In [ ]:
#Assembling Matrix
def assembling(number_of_nodes,num):
    A=np.zeros((2,number_of_nodes))
    A[0][num]=1
    A[1][num+1]=1
    return A

## Printing the Displacements into a Text File:

In [ ]:
f = open("Output_of_NLFEM_Assignement.txt", "a")

f.truncate(0)

plastic_reached = 0

In [ ]:
for i , time_step in enumerate(all_times):
    
    u_time_step = u[i-1,:,:]                                      # PREVIOUS DISPLACEMENT
    epsilon_plastic_history_time = epsilon_plastic[i-1,:,:]       # PREVIOUS PLASTIC STRAIN
    u_time_step[0,0] = time_step*epsilon_v*r_in/3                 # Ui at the First INNER Node
    epsilon_plastic_temp_storage = np.zeros_like(epsilon_plastic_history_time)
    stress_temp_storage = np.zeros_like(epsilon_plastic_history_time)
    
    ################################# NEWTON RAPHSON METHOD ##################################################
    
    for convergence_test in range(5):
        
        if convergence_test >0 :
            
            if np.amax(np.absolute(reduced_delta_u)) <= 0.005*(np.amax(np.absolute(u_time_step[1:,0]))) :
                
                print("\n !!!!!!!!!  Newton Raphson converged at Iteration number : ", convergence_test, " !!!!!!!!!!! \n")
                
                break
                
        for j in range(number_of_elements):
            
            A = assembling(number_of_nodes,j)    #Assignment Matrix
            
            u_element = np.dot(A,u_time_step)    #U_Element
            
            Kt_ele, F_int_ele, stress3D2, strain_plastic1 = elementroutine(r[j],lamda, mu,u_element,(epsilon_plastic_history_time[:,j]).reshape((3,1)),sigmaY,youngs_modulus)
            
            epsilon_plastic_temp_storage[:,j] = strain_plastic1.transpose()   #Storing the Plastic History Element Wise
            
            stress_temp_storage[:,j] = stress3D2.transpose()
            
            Kt=Kt + ((A.transpose()).dot(Kt_ele).dot(A))                      # Global Stiffness Matrix
            
            F_int=F_int+((A.transpose()).dot(F_int_ele))                      # Global Internal Force
        
        ############################ Calculation of Displacements (K * delta_U = -G) #############################
        
        reduced_Kt = Kt[1:,1:]
        
        reduced_G = F_int[1:,0]
        
        reduced_delta_u = np.linalg.inv(reduced_Kt) @ (-reduced_G)
        
        u_time_step[1:,0] = u_time_step[1:,0] + reduced_delta_u
        
    #################################### Updating the Current Displacements and Strains ########################
    
    u[i,:,:] = u_time_step
    
    epsilon_plastic[i,:,:] = epsilon_plastic_temp_storage
    
    stress_3D[i,:,:] = stress_temp_storage
    
    if np.count_nonzero(epsilon_plastic[i,:,:]) !=0 :
        
        plastic_reached += 1
        
        if plastic_reached ==1:
            print("**********************The Material has reached its Plastic Regime at iteration =", i, " ***************")
            print("epsilon_plastic = \n",epsilon_plastic[i,:,:], file = f)
    
    print("U for time step = ",time_step,"\n", u[i,:,:], file=f)
    
        
f.close()
        
        
    

## Analytical Solution for Displacements in elastic case: 

In [ ]:
r_check = np.unique(r)

u_r_elastic = r_in**3 * epsilon_v/(3*np.square(r_check))

sigma_rr_elastic = -2*youngs_modulus*epsilon_v*r_in**3 / (3*(1+poisson_ratio)*r_check**3)


## Plotting the Numerical v/s Analytical Displacements in Elastic Case:

In [ ]:
fig, ax = plt.subplots(nrows = 3, figsize=((15,10)))

ax[0].set_title('Convergence Study of Displacements in Elastic Regime', fontsize=18)
ax[0].plot(r_check,u_r_elastic, label = 'Analytical')
ax[0].plot(r_check,u[9,:,0] ,"o--", label ="Numerical")
ax[0].set_xlabel('r [$\mu m$]', fontsize=20)
ax[0].set_ylabel('U_r_Elastic [$\mu m$]', fontsize=20)
ax[0].legend()

ax[1].set_title('Convergence Study of Sigma_rr in Elastic Regime', fontsize=18)
ax[1].plot(r_check,sigma_rr_elastic, label = 'Analytical')
ax[1].plot(r_check[:-1],stress_3D[9,0,:] , "r--", label ="Numerical")
ax[1].set_xlabel('r [$\mu m$]', fontsize=20)
ax[1].set_ylabel('$\sigma_{rr}$_Elastic [MPa]', fontsize=20)
ax[1].legend()

ax[2].set_title('Non linearity of Sigma_rr w.r.t time', fontsize=18)
ax[2].plot(all_times,stress_3D[:,0,0] , '*--',label ="Numerical")
ax[2].set_xlabel('time [seconds]', fontsize=20)
ax[2].set_ylabel('$\sigma_{rr}$ [MPa]', fontsize=20)
ax[2].legend()

fig.tight_layout()
plt.show()